In [1]:
import json
import os
import time
from pandas import json_normalize
import psycopg2
from psycopg2 import sql
import numpy as np

import pandas as pd
import requests
from dotenv import load_dotenv

load_dotenv()

access_token = os.getenv("ACCESS_TOKEN")
HOST = os.getenv("HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")


# Informações de conexão com o banco de dados PostgreSQL
db_config = {
    "host": HOST,
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}

# Registra o tempo antes da execução
start_prog = time.time()

In [2]:
print(access_token)

APP_USR-6628000663056293-120415-422a24d36464c1f0729bc9ef915087a3-233632476


### Verificando dados da conta

In [3]:
url = "https://api.mercadolibre.com/users/me"

payload = {}
headers = {"Authorization": f"Bearer {access_token}"}

response = requests.request("GET", url, headers=headers, data=payload)
response = response.text

print(response)

# Encontre posição do início e fim do campo "id"
start_pos = response.find('"id"')  # Encontra onde começa "id"
end_pos = response.find(",", start_pos)  # Encontra onde termina "id"

# Extraia o valor de "id"
seller_id = response[start_pos:end_pos].split(":")[1]
seller_id = seller_id.strip().strip('"')

print(f'O valor de "seller_id" é: {seller_id}')

{"id":233632476,"nickname":"MUSICALCENTERPG","registration_date":"2016-11-05T17:35:04.000-04:00","first_name":"MUSICAL CENTER PONTA","last_name":"GROSSA EIRELI -","gender":"","country_id":"BR","email":"musicalc168@gmail.com","identification":{"number":"10885874000127","type":"CNPJ"},"address":{"address":"Estrada da Gruta 775","city":"Louveira","state":"BR-SP","zip_code":"13290431"},"phone":{"area_code":"19","extension":"","number":"38787702","verified":false},"alternative_phone":{"area_code":"","extension":"","number":""},"user_type":"brand","tags":["brand","large_seller","eshop","mshops","messages_as_seller"],"logo":null,"points":4541,"site_id":"MLB","permalink":"http://perfil.mercadolivre.com.br/MUSICALCENTERPG","seller_experience":"ADVANCED","bill_data":{"accept_credit_note":"Y"},"seller_reputation":{"level_id":"5_green","power_seller_status":"platinum","transactions":{"canceled":1553,"completed":34275,"period":"historic","ratings":{"negative":0.05,"neutral":0.03,"positive":0.92},"t

### Carregando Inventory Ids de Produtos a serem pesquisados

Esses dados podem ser encontrados na tabela *tiny_fulfillment* do banco de dados

In [4]:
conn = psycopg2.connect(**db_config)

sql_query = "SELECT * FROM items"

df_codes = pd.read_sql(sql_query, conn)

df_codes.head(1)

C:\Users\couti\AppData\Local\Temp\ipykernel_5024\2605888089.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_codes = pd.read_sql(sql_query, conn)


,ml_code,inventory_id,value_name,variation_id,status,catalog_listing,created_at,updated_at
0,MLB3778561802,AVOX05934,NaN,NaN,closed,True,2023-12-04 20:08:10.878608,2023-12-04 20:08:10.878608


In [5]:
# Verifica se há duplicatas
duplicates = df_codes.duplicated()
n_dup = duplicates.sum()

print(f"Número de duplicatas: {n_dup}")

# Encontre as duplicatas no DataFrame
duplicates = df_codes[df_codes.duplicated(keep=False)]

# Exiba as próprias duplicatas
duplicates

Número de duplicatas: 0


,ml_code,inventory_id,value_name,variation_id,status,catalog_listing,created_at,updated_at


In [6]:
# Remove as duplicatas e atualiza o DataFrame
df_codes = df_codes.drop_duplicates()

df_codes.shape

(526, 8)

In [7]:
df_codes.head(1)

,ml_code,inventory_id,value_name,variation_id,status,catalog_listing,created_at,updated_at
0,MLB3778561802,AVOX05934,NaN,NaN,closed,True,2023-12-04 20:08:10.878608,2023-12-04 20:08:10.878608


### Pegando Estoque de FulFillment

In [21]:
# removendo valores nulos
df_codes = df_codes.dropna(subset=["inventory_id"])
# df_codes["inventory_id"].value_counts()

# removendo valores nulos
df_variations = df_codes.dropna(subset=["variation_id"])
# df_variations["variation_id"].value_counts()

In [22]:
var_codes = df_codes["variation_id"].unique()
codes = df_codes["inventory_id"].unique()

print(len(codes))
print(len(var_codes))

365
172


In [25]:
import numpy as np

codes = np.delete(codes, np.where(codes == "NaN"))
var_codes = np.delete(var_codes, np.where(var_codes == "NaN"))
codes

array(['AVOX05934', 'WTHI04163', 'JLAX10536', 'JFGN34621', 'TDFV00990',
       'QYGO52123', 'ERIM51807', 'DGBX50777', 'TDFV51283', 'PKNN52962',
       'IXGP53982', 'OYUK51882', 'CCQW73335', 'DXJZ75064', 'TNXB83665',
       'JDMV76187', 'RJUL81287', 'CIKE94068', 'MENA20785', 'HXJD21466',
       'CQAQ20361', 'GLMK21133', 'ABCB20467', 'ZZGN21450', 'TXWS20511',
       'DCPP20168', 'FVTQ20181', 'HTEZ21817', 'XYHC31118', 'SCKP53673',
       'SDNY53354', 'BOWK32167', 'ZZRV30730', 'KRJP31426', 'SCVF31712',
       'MLVG32506', 'IRUD34507', 'TPDH38567', 'TLCG38735', 'PUOF38682',
       'ZXFT21237', 'JGRW38703', 'AIRQ38676', 'GPGU38258', 'ITMG31457',
       'XDTG30724', 'NWXH48678', 'QOGG45483', 'STLG62905', 'VGBX48763',
       'SKXW47953', 'KBOK48388', 'PROW53057', 'WOXA67988', 'YLXH65172',
       'CNHG47042', 'PVKY65418', 'BOWK48319', 'XYHC65769', 'XGSW66673',
       'KNPR48414', 'SKXW65690', 'HXJD67665', 'WIZE67325', 'CNHG66088',
       'CJBY66878', 'PWQU66026', 'KMCL67683', 'GMII66042', 'EDGP

In [27]:
print(len(codes))
print(len(var_codes))

365
171


In [ ]:
# unique_values = set(codes)
# num_unique_values = len(unique_values)

# print(f"Quantidade de valores únicos: {num_unique_values}")

In [15]:
unique_values = set(codes)
num_unique_values = len(unique_values)

print(f"Quantidade de valores únicos: {num_unique_values}")

Quantidade de valores únicos: 365


In [28]:
counter = 0
json_list = []

for item in codes:
    url = f"https://api.mercadolibre.com/inventories/{item}/stock/fulfillment"

    payload = {}
    headers = {"Authorization": f"Bearer {access_token}"}
    print(f"Buscando dados {counter}/{len(codes)}: {item}")

    try:
        response = requests.get(url, headers=headers, data=payload)
        response.raise_for_status()  # Lança uma exceção HTTPError para códigos de status de erro
        response_data = response.json()
        json_list.append(response_data)
        counter += 1

        if counter % 50 == 0:
            print(f"Fazendo uma pausa de 1 minuto...")
            time.sleep(60)

    except requests.exceptions.RequestException as e:
        print(f"Erro ao fazer a requisição para {url}: {e}")

    except Exception as e:
        print(f"Erro não esperado: {e}")

Buscando dados 0/365: AVOX05934
Buscando dados 1/365: WTHI04163
Buscando dados 2/365: JLAX10536
Buscando dados 3/365: JFGN34621
Buscando dados 4/365: TDFV00990
Buscando dados 5/365: QYGO52123
Buscando dados 6/365: ERIM51807
Buscando dados 7/365: DGBX50777
Buscando dados 8/365: TDFV51283
Buscando dados 9/365: PKNN52962
Buscando dados 10/365: IXGP53982
Buscando dados 11/365: OYUK51882
Buscando dados 12/365: CCQW73335
Buscando dados 13/365: DXJZ75064
Buscando dados 14/365: TNXB83665
Buscando dados 15/365: JDMV76187
Buscando dados 16/365: RJUL81287
Buscando dados 17/365: CIKE94068
Buscando dados 18/365: MENA20785
Buscando dados 19/365: HXJD21466
Buscando dados 20/365: CQAQ20361
Buscando dados 21/365: GLMK21133
Buscando dados 22/365: ABCB20467
Buscando dados 23/365: ZZGN21450
Buscando dados 24/365: TXWS20511
Buscando dados 25/365: DCPP20168
Buscando dados 26/365: FVTQ20181
Buscando dados 27/365: HTEZ21817
Buscando dados 28/365: XYHC31118
Buscando dados 29/365: SCKP53673
Buscando dados 30/36

In [31]:
file_path = "../Data/Output/lista_fulfillment_estoque.json"
with open(file_path, "w") as file:
    json.dump(json_list, file)

In [33]:
print(json_list)

[{'inventory_id': 'AVOX05934', 'total': 7, 'available_quantity': 7, 'not_available_quantity': 0, 'not_available_detail': [], 'external_references': [{'type': 'item', 'id': 'MLB2003190501'}]}, {'inventory_id': 'WTHI04163', 'total': 15, 'available_quantity': 14, 'not_available_quantity': 1, 'not_available_detail': [{'status': 'transfer', 'quantity': 1}], 'external_references': [{'type': 'item', 'id': 'MLB2000971378'}]}, {'inventory_id': 'JLAX10536', 'total': 3, 'available_quantity': 3, 'not_available_quantity': 0, 'not_available_detail': [], 'external_references': [{'type': 'item', 'id': 'MLB2004809736'}]}, {'inventory_id': 'JFGN34621', 'total': 1, 'available_quantity': 0, 'not_available_quantity': 1, 'not_available_detail': [{'status': 'lost', 'quantity': 1}], 'external_references': [{'type': 'item', 'id': 'MLB924922735'}]}, {'inventory_id': 'TDFV00990', 'total': 0, 'available_quantity': 0, 'not_available_quantity': 0, 'not_available_detail': [], 'external_references': [{'type': 'item',

### Extraindo dados

In [89]:
resultados = []

for item in json_list:
    # Extrair os valores desejados
    inventory_id = item["inventory_id"]
    total = item["total"]
    available_quantity = item["available_quantity"]
    not_available_quantity = item["not_available_quantity"]
    not_available_detail = item.get(
        "not_available_detail", []
    )  # Evita KeyError se "not_available_detail" estiver ausente
    external_references = item.get("external_references", [])

    # Inicializa variáveis para armazenar detalhes
    not_available_detail_status = None
    not_available_detail_quantity = None
    external_references_type = None
    external_references_id = None
    external_references_variation_id = None

    # Verifica se há pelo menos um item em "not_available_detail"
    if not_available_detail:
        not_available_detail_status = not_available_detail[0].get("status")
        not_available_detail_quantity = not_available_detail[0].get("quantity")

    if external_references:
        external_references_type = external_references[0].get("type")
        external_references_id = external_references[0].get("id")
        external_references_variation_id = external_references[0].get("variation_id")

    # Adicionar os resultados à lista
    resultados.append(
        {
            "inventory_id": inventory_id,
            "total": total,
            "available_quantity": available_quantity,
            "not_available_quantity": not_available_quantity,
            # "not_available_detail": not_available_detail,
            # "external_references": external_references,
            "not_available_detail_status": not_available_detail_status,
            "not_available_detail_quantity": not_available_detail_quantity,
            "external_references_type": external_references_type,
            "external_references_id": external_references_id,
            "external_references_variation_id": external_references_variation_id,
        }
    )

# Exibir os resultados
df = pd.DataFrame(resultados)
print(df.shape)
df.sample()

(364, 9)


,inventory_id,total,available_quantity,not_available_quantity,not_available_detail_status,not_available_detail_quantity,external_references_type,external_references_id,external_references_variation_id
227,RIXS53464,4,4,0,None,NaN,item,MLB1629396951,NaN


In [90]:
df.dtypes

inventory_id                         object
total                                 int64
available_quantity                    int64
not_available_quantity                int64
not_available_detail_status          object
not_available_detail_quantity       float64
external_references_type             object
external_references_id               object
external_references_variation_id    float64
dtype: object

In [91]:
# df['external_references_variation_id'].astype(str).apply(lambda x:x.rstrip('.0'))
df["external_references_variation_id"] = (
    df["external_references_variation_id"].astype(str).apply(lambda x: x.rstrip(".0"))
)
df

,inventory_id,total,available_quantity,not_available_quantity,not_available_detail_status,not_available_detail_quantity,external_references_type,external_references_id,external_references_variation_id
0,AVOX05934,7,7,0,None,NaN,item,MLB2003190501,nan
1,WTHI04163,15,14,1,transfer,1.0,item,MLB2000971378,nan
2,JLAX10536,3,3,0,None,NaN,item,MLB2004809736,nan
3,JFGN34621,1,0,1,lost,1.0,item,MLB924922735,nan
4,TDFV00990,0,0,0,None,NaN,item,MLB949771924,nan
...,...,...,...,...,...,...,...,...,...
359,IYNL59473,8,8,0,None,NaN,item,MLB3339660079,178533059695
360,ARDK78657,15,15,0,None,NaN,item,MLB3339660079,178533059697
361,IDSV51539,64,64,0,None,NaN,item,MLB3339660079,178533059699
362,BZGW51616,25,25,0,None,NaN,item,MLB3339660079,178533059701


In [92]:
df.dtypes

inventory_id                         object
total                                 int64
available_quantity                    int64
not_available_quantity                int64
not_available_detail_status          object
not_available_detail_quantity       float64
external_references_type             object
external_references_id               object
external_references_variation_id     object
dtype: object

In [94]:
df["not_available_detail_quantity"] = pd.to_numeric(
    df["not_available_detail_quantity"], errors="coerce"
).astype("Int64")
df["not_available_detail_status"] = df["not_available_detail_status"].astype(str)
df["external_references_id"] = df["external_references_id"].astype(str)
df

,inventory_id,total,available_quantity,not_available_quantity,not_available_detail_status,not_available_detail_quantity,external_references_type,external_references_id,external_references_variation_id
0,AVOX05934,7,7,0,None,<NA>,item,MLB2003190501,nan
1,WTHI04163,15,14,1,transfer,1,item,MLB2000971378,nan
2,JLAX10536,3,3,0,None,<NA>,item,MLB2004809736,nan
3,JFGN34621,1,0,1,lost,1,item,MLB924922735,nan
4,TDFV00990,0,0,0,None,<NA>,item,MLB949771924,nan
...,...,...,...,...,...,...,...,...,...
359,IYNL59473,8,8,0,None,<NA>,item,MLB3339660079,178533059695
360,ARDK78657,15,15,0,None,<NA>,item,MLB3339660079,178533059697
361,IDSV51539,64,64,0,None,<NA>,item,MLB3339660079,178533059699
362,BZGW51616,25,25,0,None,<NA>,item,MLB3339660079,178533059701


In [98]:
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

for index, row in df.iterrows():
    # Substituir valores NaN por None
    # row = row.where(pd.notna(), None)
    row = row.apply(lambda x: None if pd.isna(x) else x)

    insert_query = sql.SQL(
        """
        INSERT INTO fulfillment_stock_hist (ml_inventory_id, available_quantity, detail_status, detail_quantity, references_id, references_variation_id)
        VALUES (%s, %s, %s, %s, %s, %s)
        """
    )
    cursor.execute(
        insert_query,
        (
            row["inventory_id"],
            row["available_quantity"],
            row["not_available_detail_status"],
            row["not_available_detail_quantity"],
            row["external_references_id"],
            row["external_references_variation_id"],
        ),
    )
    # print(insert_query)

conn.commit()

cursor.close()
conn.close()

print("Dados inseridos com sucesso!")

SQL('\n        INSERT INTO fulfillment_stock_hist (ml_inventory_id, available_quantity, detail_status, detail_quantity, references_id, references_variation_id)\n        VALUES (%s, %s, %s, %s, %s, %s)\n        ')
SQL('\n        INSERT INTO fulfillment_stock_hist (ml_inventory_id, available_quantity, detail_status, detail_quantity, references_id, references_variation_id)\n        VALUES (%s, %s, %s, %s, %s, %s)\n        ')
SQL('\n        INSERT INTO fulfillment_stock_hist (ml_inventory_id, available_quantity, detail_status, detail_quantity, references_id, references_variation_id)\n        VALUES (%s, %s, %s, %s, %s, %s)\n        ')
SQL('\n        INSERT INTO fulfillment_stock_hist (ml_inventory_id, available_quantity, detail_status, detail_quantity, references_id, references_variation_id)\n        VALUES (%s, %s, %s, %s, %s, %s)\n        ')
SQL('\n        INSERT INTO fulfillment_stock_hist (ml_inventory_id, available_quantity, detail_status, detail_quantity, references_id, references_va

In [99]:
df

,inventory_id,total,available_quantity,not_available_quantity,not_available_detail_status,not_available_detail_quantity,external_references_type,external_references_id,external_references_variation_id
0,AVOX05934,7,7,0,None,<NA>,item,MLB2003190501,nan
1,WTHI04163,15,14,1,transfer,1,item,MLB2000971378,nan
2,JLAX10536,3,3,0,None,<NA>,item,MLB2004809736,nan
3,JFGN34621,1,0,1,lost,1,item,MLB924922735,nan
4,TDFV00990,0,0,0,None,<NA>,item,MLB949771924,nan
...,...,...,...,...,...,...,...,...,...
359,IYNL59473,8,8,0,None,<NA>,item,MLB3339660079,178533059695
360,ARDK78657,15,15,0,None,<NA>,item,MLB3339660079,178533059697
361,IDSV51539,64,64,0,None,<NA>,item,MLB3339660079,178533059699
362,BZGW51616,25,25,0,None,<NA>,item,MLB3339660079,178533059701


In [4]:
db_config = {
    "host": "backup2.cmenzobfpepq.sa-east-1.rds.amazonaws.com",
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}
# Buscando histórico de vendas na tabela ml_orders_hist para o período definido
try:
    conn = psycopg2.connect(**db_config)

    # Construa a consulta SQL com a condição de data
    sql_query = f"SELECT * FROM fulfillment_stock_hist"
    print(sql_query)
    # Execute a consulta e leia os dados em um DataFrame
    df = pd.read_sql(sql_query, conn)

except psycopg2.Error as e:
    print(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")
    # logger.error(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")

except Exception as e:
    print(f"Erro ao consultar ml_orders_hist: {e}")
    # logger.error(f"Erro ao consultar ml_orders_hist: {e}")

finally:
    if conn is not None:
        conn.close()

SELECT * FROM fulfillment_stock_hist


C:\Users\couti\AppData\Local\Temp\ipykernel_4532\3790987738.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)


In [5]:
df

,ml_inventory_id,available_quantity,created_at
0,DSGP06967,6,2023-11-23 11:43:27.867518
1,DSGP06979,7,2023-11-23 11:43:27.867518
2,JJSZ06277,48,2023-11-23 11:43:27.867518
3,DGCL82617,13,2023-11-23 11:43:27.867518
4,GCYF86172,12,2023-11-23 11:43:27.867518
...,...,...,...
45077,ERIM51807,0,2023-12-04 00:01:03.041159
45078,TDFV51283,0,2023-12-04 00:01:03.041159
45079,OYUK51882,0,2023-12-04 00:01:03.041159
45080,BLHH53768,14,2023-12-04 00:01:03.041159


In [8]:
db_config = {
    "host": "mcenter.cmenzobfpepq.sa-east-1.rds.amazonaws.com",
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}

conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

for index, row in df.iterrows():
    # Substituir valores NaN por None
    # row = row.where(pd.notna(), None)
    # row = row.apply(lambda x: None if pd.isna(x) else x)

    insert_query = sql.SQL(
        """
        INSERT INTO fulfillment_stock_hist (ml_inventory_id, available_quantity, created_at)
        
        VALUES (%s, %s, %s)
        """
    )
    cursor.execute(
        insert_query,
        (
            row["ml_inventory_id"],
            row["available_quantity"],
            row["created_at"],
            # row["not_available_detail_status"],
            # row["not_available_detail_quantity"],
            # row["external_references_id"],
            # row["external_references_variation_id"],
        ),
    )
    # print(insert_query)

conn.commit()

cursor.close()
conn.close()

print("Dados inseridos com sucesso!")

Dados inseridos com sucesso!


In [21]:
### AMBIENTE DE TESTES ###
### Repetindo os dados com dias diferentes
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()
import datetime

custom_created_at = datetime.datetime(2023, 11, 20, 1, 0, 0)

for _ in range(30):  # Repetir 30 vezes para adicionar dados com datas diferentes
    for index, row in df_full.iterrows():  # preenchendo db com dados de estoque
        insert_query = sql.SQL(
            """
                INSERT INTO fulfillment_stock_hist (ml_inventory_id, available_quantity, created_at)
                VALUES (%s, %s, %s)
            """
        )
        cursor.execute(
            insert_query,
            (row["inventory_id"], row["available_quantity"], custom_created_at),
        )
    custom_created_at += datetime.timedelta(days=-1)


conn.commit()

cursor.close()
conn.close()

print("Dados inseridos com sucesso!")

Dados inseridos com sucesso!
